In [3]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

In [6]:
data = pd.read_csv('./Data/OutputDatos.csv')
data.head()

,tconst,titleType,originalTitle,startYear,runtimeMinutes,numVotes,ordering,region,main_genre,secondary_genre
0,103064,movie,Terminator 2: Judgment Day,1991,137,1128166,17,US,Action,Sci-Fi
1,231050,tvSeries,ToransufÃ´mÃ¢: KÃ¢robotto,2000,22,577,16,US,Action,Adventure
2,253474,movie,The Pianist,2002,150,860569,7,US,Biography,Drama
3,369339,movie,Collateral,2004,120,417391,6,US,Action,Crime
4,439547,movie,2 Filhos de Francisco: A HistÃ³ria de ZezÃ© di...,2005,132,5946,5,US,Biography,Drama
